In [2]:
import geopandas as gpd

# 1) Read your Lombardy polygon
roi = gpd.read_file("F:/GeoinformaticsProject/Data/Lombardy.geojson")

# 2) Reproject to the target CRS
roi = roi.to_crs("EPSG:32632")

# Extract the geometries as GeoJSON-like dicts
shapes = [feature["geometry"] for feature in roi.__geo_interface__['features']]


In [4]:
import os
import rasterio
from rasterio.mask import mask

# ── 0. Your Lombardy shapes (from earlier) ────────────────────────────────
# shapes = [ … ]  # the list of GeoJSON geometries from your reprojected ROI

# ── 1. Weekly source folders ──────────────────────────────────────────────
weekly_dirs = {
    'MODIS': r"F:/GeoinformaticsProject/Data/Processed/WEEKLY/MODIS",
    'GFSC' : r"E:/ALIGNED/ToMODIS/GFSC",
    'S2'   : r"E:/ALIGNED/ToMODIS/S2",
    'S3'   : r"E:/ALIGNED/ToMODIS/S3"
}

common_weeks = [
    '2022_W07', '2022_W09', '2022_W11',
    '2022_W44', '2022_W49', '2022_W50',
    '2023_W04', '2023_W06', '2023_W08',
    '2023_W09', '2023_W10', '2023_W11',
    '2023_W13', '2023_W14', '2023_W15', '2023_W17'
]

# ── 2. Clipped output root ─────────────────────────────────────────────────
clipped_root = r"F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED"
os.makedirs(clipped_root, exist_ok=True)

# ── 3. Loop over products & weeks ──────────────────────────────────────────
for prod, src_folder in weekly_dirs.items():
    dst_folder = os.path.join(clipped_root, prod)
    os.makedirs(dst_folder, exist_ok=True)

    for week in common_weeks:
        src_path = os.path.join(src_folder, f"{week}.tif")
        dst_path = os.path.join(dst_folder,   f"{week}.tif")

        # Skip if the source TIFF doesn’t exist
        if not os.path.exists(src_path):
            continue

        # Read & mask
        with rasterio.open(src_path) as src:
            out_img, out_transform = mask(src, shapes, crop=True)
            out_meta = src.meta.copy()

        # Update metadata (preserve nodata)
        out_meta.update({
            "driver"   : "GTiff",
            "height"   : out_img.shape[1],
            "width"    : out_img.shape[2],
            "transform": out_transform,
            "nodata"   : src.meta["nodata"]
        })

        # Write clipped file
        with rasterio.open(dst_path, "w", **out_meta) as dst:
            dst.write(out_img)

        print(f"🏷  Clipped {prod} {week} → {dst_path}")


🏷  Clipped MODIS 2022_W07 → F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED\MODIS\2022_W07.tif
🏷  Clipped MODIS 2022_W09 → F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED\MODIS\2022_W09.tif
🏷  Clipped MODIS 2022_W11 → F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED\MODIS\2022_W11.tif
🏷  Clipped MODIS 2022_W44 → F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED\MODIS\2022_W44.tif
🏷  Clipped MODIS 2022_W49 → F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED\MODIS\2022_W49.tif
🏷  Clipped MODIS 2022_W50 → F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED\MODIS\2022_W50.tif
🏷  Clipped MODIS 2023_W04 → F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED\MODIS\2023_W04.tif
🏷  Clipped MODIS 2023_W06 → F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED\MODIS\2023_W06.tif
🏷  Clipped MODIS 2023_W08 → F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED\MODIS\2023_W08.tif
🏷  Clipped MODIS 2023_W09 → F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED

In [1]:
from snow_processing import compute_weekly_statistics

# 1) Define where your clipped files live and which products to include
clipped_root = r"F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED"
products     = ["MODIS", "GFSC", "S2", "S3"]

# 2) Run the stats function
df_stats = compute_weekly_statistics(clipped_root, products, pixel_size=20.0)

# 3) Inspect the first few rows
print(df_stats.head())


f:\GeoinformaticsProject\Code\snow_processing.py:47: SyntaxWarning: invalid escape sequence '\M'
  """


  product      week  total_pixels  snow_pixels  missing_pixels  snow_area_km2  \
0    GFSC  2022_W07      59653061      4649496        66142553      1859.7984   
1   MODIS  2022_W07      59653061     44011842        66142553     17604.7368   
2      S2  2022_W07      59653061      8667788        66142553      3467.1152   
3      S3  2022_W07      59653061     12428916        66142553      4971.5664   
4    GFSC  2022_W09      59653061      2273490        66142553       909.3960   

   coverage_pct  
0      7.794229  
1     73.779688  
2     14.530332  
3     20.835337  
4      3.811187  


In [1]:
from snow_processing import compute_pairwise_agreement

# 1) Point at your **clipped** folders, not the aligned ones:
clipped_root  = r"F:/GeoinformaticsProject/Data/Processed/WEEKLY_CLIPPED"
products      = ["MODIS", "GFSC", "S2", "S3"]
common_weeks  = [
    '2022_W07', '2022_W09', '2022_W11',
    '2022_W44', '2022_W49', '2022_W50',
    '2023_W04', '2023_W06', '2023_W08',
    '2023_W09', '2023_W10', '2023_W11',
    '2023_W13', '2023_W14', '2023_W15', '2023_W17'
]

# 2) Run agreement on clipped Lombardy‐only files
df_agree = compute_pairwise_agreement(clipped_root, products, common_weeks)

# 3) Verify it’s non‐empty and has the “week” column
#print(df_agree.shape)
#print(df_agree.columns)
print(df_agree.head())


f:\GeoinformaticsProject\Code\snow_processing.py:47: SyntaxWarning: invalid escape sequence '\M'
  """


       week  prod1 prod2  agreement_pct  total_pixels  agree_pixels  \
0  2022_W07   GFSC    S2      37.721516       8667788       3269621   
1  2022_W07   GFSC    S3      19.846011      12428916       2466644   
2  2022_W07  MODIS  GFSC       1.924485      59653061       1148014   
3  2022_W07  MODIS    S2      10.020653       8667788        868569   
4  2022_W07  MODIS    S3      14.947080      12428916       1857760   

   p1_only_pixels  p2_only_pixels  p1_only_area_km2  p2_only_area_km2  \
0         1379875         5398167          551.9500         2159.2668   
1         2182852         9962272          873.1408         3984.9088   
2        42863828         3501482        17145.5312         1400.5928   
3        43143273         7799219        17257.3092         3119.6876   
4        42154082        10571156        16861.6328         4228.4624   

   area_bias_km2  
0      1607.3168  
1      3111.7680  
2    -15744.9384  
3    -14137.6216  
4    -12633.1704  
